<a href="https://colab.research.google.com/github/Ixiondbz/CSE499/blob/main/NEW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install dependencies and 
!pip install albumentations==0.4.6
!pip install pycocotools --quiet

# Clone TorchVision repo and copy helper files
!git clone https://github.com/pytorch/vision.git
%cd vision
!git checkout v0.3.0
%cd ..
!cp vision/references/detection/utils.py ./
!cp vision/references/detection/transforms.py ./
!cp vision/references/detection/coco_eval.py ./
!cp vision/references/detection/engine.py ./
!cp vision/references/detection/coco_utils.py ./


# basic python and ML Libraries
import os
import random
import numpy as np
import pandas as pd

# for ignoring warnings
import warnings
warnings.filterwarnings('ignore')

# We will be reading images using OpenCV
import cv2

# matplotlib for visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# torchvision libraries
import torch
import torchvision
from torchvision import transforms as torchtrans  
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# helper libraries
from engine import train_one_epoch, evaluate
import utils
import transforms as T

# for image augmentations
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

!apt-get install python3-openslide
from openslide import open_slide

import plotly.express as px

from torchvision.models.detection import FasterRCNN
from torch.utils.data import DataLoader, Dataset

     |████████████████████████████████| 117 kB 4.0 MB/s 
     |████████████████████████████████| 948 kB 14.5 MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-py3-none-any.whl size=65172 sha256=f8f93763d28d3011e5a729be997ad474cde3acea668705b4793d8cb4f9c8f918
  Stored in directory: /root/.cache/pip/wheels/cf/34/0f/cb2a5f93561a181a4bcc84847ad6aaceea8b5a3127469616cc
Successfully built albumentations
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12
Cloning into 'vision'...
remote: Enumerating objects: 66925, done.
remote: Counting objects: 100% (1986/1986), done.
remote: Compressing objects: 100% (209/209), done.
remote: Total 66925 (delta 1783), reused 1935 (delta 1767), pack-reused

In [2]:
import json
from pathlib import Path

from google.colab import drive
drive.mount('/drive')

folder = "MyDrive/MIDOG_Challenge"
midog_folder = Path("/drive") / Path(folder)
print(list(midog_folder.glob("*.*")))

image_folder = Path("/content/drive/MyDrive/CSE499 Project/train_images")

hamamatsu_rx_ids = list(range(0, 51))
hamamatsu_360_ids = list(range(51, 101))
aperio_ids = list(range(101, 151))
leica_ids = list(range(151, 201))


annotation_file = midog_folder / "MIDOG.json"
rows = []

Mounted at /drive
[PosixPath('/drive/MyDrive/MIDOG_Challenge/MIDOG.sqlite'), PosixPath('/drive/MyDrive/MIDOG_Challenge/MIDOG.json')]


In [3]:
with open(annotation_file) as f:
    data = json.load(f)

    categories = {1: 'mitotic figure', 2: 'hard negative'}

    for row in data["images"]:
        file_name = row["file_name"]
        image_id = row["id"]
        width = row["width"]
        height = row["height"]

        scanner  = "Hamamatsu XR"
        if image_id in hamamatsu_360_ids:
            scanner  = "Hamamatsu S360"
        if image_id in aperio_ids:
            scanner  = "Aperio CS"
        if image_id in leica_ids:
            scanner  = "Leica GT450"
         
        for annotation in [anno for anno in data['annotations'] if anno["image_id"] == image_id]:
            box = annotation["bbox"]
            cat = categories[annotation["category_id"]]

            rows.append([file_name, image_id, width, height, box, cat, scanner])
        
df = pd.DataFrame(rows, columns=["file_name", "image_id", "width", "height", "box", "cat", "scanner"])

In [4]:
df[df['cat']=='hard negative'].index

Int64Index([   0,    1,    2,    3,    4,    7,   10,   11,   14,   15,
            ...
            4420, 4422, 4423, 4424, 4425, 4426, 4430, 4432, 4433, 4434],
           dtype='int64', length=2714)

In [5]:
df.drop(df[df['cat']=='hard negative'].index,inplace=True)

In [6]:
df

,file_name,image_id,width,height,box,cat,scanner
5,002.tiff,2,7215,5412,"[4397, 191, 4447, 241]",mitotic figure,Hamamatsu XR
6,002.tiff,2,7215,5412,"[1842, 1558, 1892, 1608]",mitotic figure,Hamamatsu XR
8,002.tiff,2,7215,5412,"[5319, 3252, 5369, 3302]",mitotic figure,Hamamatsu XR
9,002.tiff,2,7215,5412,"[6242, 3193, 6292, 3243]",mitotic figure,Hamamatsu XR
12,002.tiff,2,7215,5412,"[4636, 4294, 4686, 4344]",mitotic figure,Hamamatsu XR
...,...,...,...,...,...,...,...
4421,150.tiff,150,6467,4862,"[5373, 2398, 5423, 2448]",mitotic figure,Aperio CS
4427,150.tiff,150,6467,4862,"[3698, 3258, 3748, 3308]",mitotic figure,Aperio CS
4428,150.tiff,150,6467,4862,"[308, 4254, 358, 4304]",mitotic figure,Aperio CS
4429,150.tiff,150,6467,4862,"[553, 4394, 603, 4444]",mitotic figure,Aperio CS


In [7]:
dataset_1 = df[df['scanner']=='Hamamatsu XR']
dataset_2 = df[df['scanner']=='Hamamatsu S360']
dataset_3 = df[df['scanner']=='Aperio CS']
#dataset_4 = df[df['scanner']=='Leica GT450']

In [8]:
len(dataset_1['image_id'].unique())+len(dataset_2['image_id'].unique())+len(dataset_3['image_id'].unique())

135

In [9]:
image_folder = Path("/content/drive/MyDrive/CSE499 Project/train_images")

In [10]:
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
# img_id = 2
# thumbail_size_big = 1024
# slide = open_slide( str(image_folder / f"{img_id:03d}.tiff") )

# image = slide.get_thumbnail((1443, 1082))

# fig = px.imshow(image)

# scale_x = slide.level_dimensions[0][0] / image.size[0]
# scale_y = slide.level_dimensions[0][1] / image.size[1]

# for id, anno in dataset_1[dataset_1["image_id"] == img_id].iterrows():

#     x0, y0, x1, y1 = anno.box[0] / scale_x, anno.box[1] / scale_y, anno.box[2] / scale_x, anno.box[3] / scale_y


#     fig.add_shape(
#         type='rect',
#         x0=x0, x1=x1, y0=y0, y1=y1,
#         xref='x', yref='y',
#         line_color='red' if "mitotic" in anno["cat"] else "blue"
#     )

# fig.update_layout(
#     autosize=False,
#     width=image.size[0],
#     height=image.size[1],
#     )

# fig.show()    

In [62]:
class MitosisImagesDataset(torch.utils.data.Dataset):
  def __init__(self, files_dir, width, height, dataset,transforms=None):
    self.transforms = transforms
    self.files_dir  = files_dir
    self.height     = height
    self.width      = width

    self.imgs = [image for image in sorted(os.listdir(files_dir)) if image[-5:]=='.tiff' and image in dataset_1['file_name'].values]
    
    #self.classes = [_, 'mitotic figure']

  def __getitem__(self, index):
    img_name   = self.imgs[index]
    image_path = os.path.join(self.files_dir, img_name)

    img     = cv2.imread(image_path)
    img     = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
    img     = cv2.resize(img, (self.width, self.height), cv2.INTER_AREA)

    img    /= 255.0
    
    
    dataframe = dataset_1[dataset_1['file_name']==img_name]
    
    boxes = []
    labels = []

    wt = img.shape[1]
    ht = img.shape[0]
    
    for index in dataframe.index:
      labels.append(1)
      
      List = dataframe['box'][index]
      
      scale_x = 7215/self.width
      scale_y = 5412/self.height
  
      List[0]=List[0]/scale_x
      List[2]=List[2]/scale_x
      List[1]=List[1]/scale_y
      List[3]=List[3]/scale_y

      xmin = List[0]
      xmax = List[2]
      ymin = List[1]
      ymax = List[3]
        
      xmin_corr = (xmin)
      xmax_corr = (xmax)
      ymin_corr = (ymin)
      ymax_corr = (ymax)
          
      boxes.append([xmin_corr, ymin_corr, xmax_corr, ymax_corr])
    


    boxes = torch.as_tensor(boxes, dtype=torch.float32)
    labels = torch.as_tensor(labels, dtype=torch.int64)
    area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
    #area = torch.as_tensor(area, dtype=torch.float32)
    iscrowd = torch.zeros((boxes.shape[0],), dtype=torch.int64)
    
    target = {}
    target["boxes"] = boxes
    target["labels"] = labels
    target["image_id"] = torch.tensor([index])
    target["area"] = area
    target["iscrowd"] = iscrowd
    
    img = torchvision.transforms.ToTensor()(img)

    return img, target
    
  def __len__(self):
    return len(self.imgs)

In [13]:
# Send train=True for training transforms and False for val/test transforms
def get_transform(train):
  if train:
    return A.Compose(
      [
        A.HorizontalFlip(0.5),
        # ToTensorV2 converts image to pytorch tensor without div by 255
        ToTensorV2(p=1.0) 
      ],
      bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']}
    )
  else:
    return A.Compose(
      [ToTensorV2(p=1.0)],
      bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']}
    )

In [61]:
files_dir = '/content/drive/MyDrive/CSE499 Project/train_images'
test_dir = '/content/drive/MyDrive/CSE499 Project/test_images'

In [63]:
dataset =      MitosisImagesDataset(files_dir, 480, 480,dataset_1,transforms=get_transform(train=False))
dataset_test = MitosisImagesDataset(files_dir, 480, 480,dataset_1,transforms=get_transform(train=False))

In [16]:
dataset[0]

(tensor([[[0.8043, 0.9302, 0.9069,  ..., 0.5588, 0.8601, 0.8914],
          [0.7941, 0.8958, 0.9083,  ..., 0.9020, 0.7432, 0.9166],
          [0.7131, 0.9394, 0.9578,  ..., 0.8909, 0.8853, 0.8485],
          ...,
          [0.8045, 0.9125, 0.9415,  ..., 0.8179, 0.7040, 0.8316],
          [0.9068, 0.8735, 0.9395,  ..., 0.7585, 0.6956, 0.8213],
          [0.8880, 0.8307, 0.9266,  ..., 0.8119, 0.8225, 0.8427]],
 
         [[0.5674, 0.8692, 0.7097,  ..., 0.3130, 0.5027, 0.6179],
          [0.5072, 0.8491, 0.8123,  ..., 0.5204, 0.4074, 0.7700],
          [0.4834, 0.9149, 0.9158,  ..., 0.5005, 0.6711, 0.6287],
          ...,
          [0.6088, 0.7011, 0.8237,  ..., 0.5412, 0.4373, 0.6157],
          [0.7490, 0.7370, 0.8159,  ..., 0.5175, 0.4052, 0.6148],
          [0.7634, 0.6891, 0.8283,  ..., 0.5844, 0.5160, 0.5681]],
 
         [[0.7720, 0.9122, 0.8528,  ..., 0.6492, 0.7754, 0.8516],
          [0.7549, 0.8960, 0.9328,  ..., 0.8478, 0.6626, 0.8842],
          [0.7643, 0.9312, 0.9435,  ...,

In [64]:
train_dataset =      MitosisImagesDataset(files_dir, 480, 480,dataset_1,transforms=get_transform(train=False))

def collate_fn(batch):
    return tuple(zip(*batch))

train_data_loader  = DataLoader(train_dataset,batch_size=10,shuffle=True,num_workers=4,collate_fn=utils.collate_fn)

In [65]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [66]:
device

device(type='cuda')

In [20]:
# from matplotlib import pyplot as plt 

# images,targets = next(iter(train_data_loader))
# images = list(image.to(device) for image in images)
# targets = [{k: v.to(device) for k,v in t.items()} for t in targets]

# boxes = targets[0]['boxes'].cpu().numpy().astype(np.int32)
# img = images[0].permute(1,2,0).cpu().numpy()
# fig,ax = plt.subplots(1, 1, figsize=(50,30))

# for box in boxes:
#   cv2.rectangle(img,
#                 (box[0],box[1]),
#                 (box[2],box[3]),
#                 (220,0,0), 1)

# ax.set_axis_off()
# ax.imshow(img)

In [67]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
num_classes=2
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes) 

In [68]:
model = model.to(device)

In [69]:
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
num_epochs = 20

In [ ]:
itr = 1

for epoch in range(num_epochs):
  for images,targets in train_data_loader:
    images = list(image.to(device) for image in images)
    targets = [{k: v.to(device) for k,v in t.items()} for t in targets]

    loss_dict = model(images,targets)

    losses = sum(loss for loss in loss_dict.values())
    loss_value = losses.item()

    optimizer.zero_grad()
    losses.backward()
    optimizer.step()

    if itr % 50 == 0:
      print(f"Iteration #{itr} loss: {loss_value}")

    itr+=1

    lr_scheduler.step()

  print(f"Epoch #{epoch} loss: {loss_value}")

Epoch #0 loss: 1.278481364250183
Epoch #1 loss: 1.038503646850586
Epoch #2 loss: 0.7167458534240723
Epoch #3 loss: 1.0711729526519775
Epoch #4 loss: 0.8949969410896301
Epoch #5 loss: 0.8229814767837524
Epoch #6 loss: 0.8153839707374573
Epoch #7 loss: 0.9438090324401855
Epoch #8 loss: 0.7520048022270203
Iteration #50 loss: 1.083060622215271
Epoch #9 loss: 1.083060622215271
Epoch #10 loss: 0.8516127467155457
Epoch #11 loss: 0.8825467824935913
Epoch #12 loss: 1.0694729089736938
Epoch #13 loss: 0.6851437091827393
Epoch #14 loss: 1.135145664215088
Epoch #15 loss: 0.8707451820373535
Epoch #16 loss: 0.98777174949646
Epoch #17 loss: 0.7367779016494751
Epoch #18 loss: 0.7707308530807495
Iteration #100 loss: 0.9232560396194458
Epoch #19 loss: 0.9232560396194458


In [70]:
%cd '/content/drive/MyDrive/CSE499 Project/modelSaves'

/content/drive/.shortcut-targets-by-id/1M-kTVs7g43FucvCjcPvKwiR8r1e1qtHe/CSE499 Project/modelSaves


In [ ]:
torch.save(model.state_dict(), 'model.pth')

torch.save({
'epoch':epoch,
'model_state_dict':model.state_dict(),
'optimizer_state_dict':optimizer.state_dict(),
},'ckpt.pth')

In [ ]:
num_epochs = 20

for epoch in range(num_epochs):
    # training for one epoch
    train_one_epoch(model, optimizer, train_data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    #evaluate(model, data_loader_test, device=device)

Epoch: [0]  [0/5]  eta: 0:12:31  lr: 0.001254  loss: 8.2572 (8.2572)  loss_classifier: 0.3805 (0.3805)  loss_box_reg: 0.0200 (0.0200)  loss_objectness: 6.9114 (6.9114)  loss_rpn_box_reg: 0.9453 (0.9453)  time: 150.3408  data: 147.3893  max mem: 7153
Epoch: [0]  [4/5]  eta: 0:00:32  lr: 0.005000  loss: 2.9613 (4.7788)  loss_classifier: 0.3372 (0.2733)  loss_box_reg: 0.0221 (0.0222)  loss_objectness: 1.9018 (3.5351)  loss_rpn_box_reg: 0.9453 (0.9481)  time: 32.9056  data: 31.4005  max mem: 7419
Epoch: [0] Total time: 0:02:44 (32.9518 s / it)
Epoch: [1]  [0/5]  eta: 0:04:03  lr: 0.005000  loss: 2.1186 (2.1186)  loss_classifier: 0.1205 (0.1205)  loss_box_reg: 0.0177 (0.0177)  loss_objectness: 0.4951 (0.4951)  loss_rpn_box_reg: 1.4852 (1.4852)  time: 48.7364  data: 47.3943  max mem: 7419
Epoch: [1]  [4/5]  eta: 0:00:11  lr: 0.005000  loss: 3.3542 (4.8970)  loss_classifier: 0.2216 (0.4149)  loss_box_reg: 0.0177 (0.0429)  loss_objectness: 1.3374 (2.4685)  loss_rpn_box_reg: 1.6719 (1.9707)  ti

In [71]:
model.load_state_dict(torch.load('./model.pth'))
model = model.to(device)

In [72]:
test_dataset =      MitosisImagesDataset(files_dir, 480, 480,dataset_1,transforms=get_transform(train=False))

torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()

test_split = 0.2
tsize = int(len(test_dataset)*test_split)

test_dataset = torch.utils.data.Subset(test_dataset, indices[-tsize:])

test_data_loader = torch.utils.data.DataLoader(
  test_dataset,
  batch_size=10,
  shuffle=False,
  num_workers=4,
  collate_fn=utils.collate_fn,
)

In [73]:
from coco_utils import get_coco_api_from_dataset
from coco_eval import CocoEvaluator
import time

def _get_iou_types(model):
    model_without_ddp = model
    if isinstance(model, torch.nn.parallel.DistributedDataParallel):
        model_without_ddp = model.module
    iou_types = ["bbox"]
    if isinstance(model_without_ddp, torchvision.models.detection.MaskRCNN):
        iou_types.append("segm")
    if isinstance(model_without_ddp, torchvision.models.detection.KeypointRCNN):
        iou_types.append("keypoints")
    return iou_types


In [74]:
def evaluate(model, data_loader, device):
    n_threads = torch.get_num_threads()
    # FIXME remove this and make paste_masks_in_image run on the GPU
    torch.set_num_threads(1)
    cpu_device = torch.device("cpu")
    model.eval()
    metric_logger = utils.MetricLogger(delimiter="  ")
    header = "Test:"

    coco = get_coco_api_from_dataset(data_loader.dataset)
    iou_types = _get_iou_types(model)
    coco_evaluator = CocoEvaluator(coco, iou_types)

    for images, targets in metric_logger.log_every(data_loader, 100, header):
        images = list(img.to(device) for img in images)

        if torch.cuda.is_available():
            torch.cuda.synchronize()
        model_time = time.time()
        outputs = model(images)

        outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
        model_time = time.time() - model_time

        res = {target["image_id"].item(): output for target, output in zip(targets, outputs)}
        evaluator_time = time.time()
        coco_evaluator.update(res)
        evaluator_time = time.time() - evaluator_time
        metric_logger.update(model_time=model_time, evaluator_time=evaluator_time)

    # gather the stats from all processes
    metric_logger.synchronize_between_processes()
    print("Averaged stats:", metric_logger)
    coco_evaluator.synchronize_between_processes()

    # accumulate predictions from all images
    coco_evaluator.accumulate()
    coco_evaluator.summarize()
    torch.set_num_threads(n_threads)
    return coco_evaluator

In [36]:
evaluate(model,test_data_loader,device)

creating index...
index created!
Test:  [0/1]  eta: 0:00:05  model_time: 0.4503 (0.4503)  evaluator_time: 0.0102 (0.0102)  time: 5.4460  data: 4.9768  max mem: 4198
Test: Total time: 0:00:05 (5.5839 s / it)
Averaged stats: model_time: 0.4503 (0.4503)  evaluator_time: 0.0102 (0.0102)
Accumulating evaluation results...
DONE (t=0.00s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.000
 Avera

In [75]:
_get_iou_types(model)

['bbox']

In [76]:
metric_logger = utils.MetricLogger(delimiter="  ")

In [ ]:
model.eval()

In [92]:
for images,targets in metric_logger.log_every(test_data_loader, 20, "Test:"):
  images = list(img.to(device) for img in images)
  outputs = model(images)

Test:  [0/1]  eta: 0:00:05    time: 5.3430  data: 4.9099  max mem: 14010
Test: Total time: 0:00:05 (5.4625 s / it)


In [93]:
images[0]

tensor([[[0.7351, 0.9058, 0.7882,  ..., 0.8693, 0.4645, 0.4778],
         [0.8875, 0.8364, 0.9017,  ..., 0.7611, 0.8373, 0.3781],
         [0.8055, 0.8575, 0.5814,  ..., 0.7822, 0.8080, 0.6478],
         ...,
         [0.3841, 0.5094, 0.5809,  ..., 0.8873, 0.8497, 0.8162],
         [0.6801, 0.6142, 0.5937,  ..., 0.9629, 0.5700, 0.8521],
         [0.5729, 0.5166, 0.8065,  ..., 0.9381, 0.9365, 0.9307]],

        [[0.3364, 0.6407, 0.4384,  ..., 0.6001, 0.2082, 0.2010],
         [0.4327, 0.4909, 0.5490,  ..., 0.4419, 0.5520, 0.1527],
         [0.3593, 0.4020, 0.3353,  ..., 0.4237, 0.4602, 0.3460],
         ...,
         [0.2141, 0.3200, 0.3401,  ..., 0.7820, 0.3727, 0.3667],
         [0.4003, 0.3675, 0.3585,  ..., 0.8812, 0.2844, 0.4824],
         [0.3488, 0.3206, 0.4810,  ..., 0.7214, 0.7704, 0.7469]],

        [[0.7784, 0.7737, 0.8183,  ..., 0.8204, 0.5750, 0.5854],
         [0.6888, 0.7016, 0.7914,  ..., 0.7611, 0.7707, 0.4502],
         [0.7361, 0.7501, 0.6398,  ..., 0.7438, 0.7694, 0.

In [94]:
outputs[0]

{'boxes': tensor([], device='cuda:0', size=(0, 4), grad_fn=<StackBackward0>),
 'labels': tensor([], device='cuda:0', dtype=torch.int64),
 'scores': tensor([], device='cuda:0', grad_fn=<IndexBackward0>)}